 ## Vendor Performance Analytics-Data Preparation


**Notebook 01: Data Extraction & Vendor Sales Summary Creation**

**1.  Key Objectives**
This notebook prepares and cleans data for the Vendor Performance Analytics project.
It performs the following tasks:
* Connects to the DuckDB database and Google Drive.
* Loads raw CSV files (purchases, sales, vendor_invoice, purchase_prices) into DuckDB.
*  Inspect and clean data for missing values, whitespace issues, and data types.
* Creates aggregated tables and metrics to evaluate vendor performance.
* Compute vendor-level metrics:
   * Total Purchase Quantity & Dollars
   * Total Sales Quantity, Dollars & Price
   * Total Excise Tax
   * Freight Cost
   * Gross Profit & Profit Margin
   * Stock Turnover
   * Sales-to-Purchase Ratio
* Generates a final vendor_sales_summary table combining purchase, sales, and freight data.
* Exports the cleaned summary as a CSV for further analysis and visualization.

**2. Technologies Used**

* Python
* DuckDB / SQL
* Pandas

**3. Output**

A clean, analysis-ready table:

➡️ vendor_sales_summary

This table serves as the foundation for all further visualizations and insights in the next notebook.

In [1]:
# 1. Mount Google Drive
from google.colab import drive
import duckdb
import os
import shutil

drive.mount('/content/drive', force_remount=True)

# 2. Paths
folder = "/content/drive/MyDrive/data/data"        # Folder with CSVs
final_db_path = "/content/drive/MyDrive/data/data.duckdb"  # Final DB in Drive
temp_db_path = "/content/data_temp.duckdb"        # Temporary DB in Colab

# 3. Connect to temporary DuckDB
con = duckdb.connect(temp_db_path)
print("Connected to temporary DuckDB:", temp_db_path)

# 4. Loop through CSV files and ingest into DuckDB
for file in os.listdir(folder):
    if file.endswith(".csv"):
        path = os.path.join(folder, file)
        # Make SQL-safe table name: replace spaces/special chars with underscores
        table_name = os.path.splitext(file)[0].replace(" ", "_").replace("-", "_")
        print(f"Processing: {file} -> Table: {table_name}")

        # Create table from CSV
        con.execute(f"""
            CREATE OR REPLACE TABLE {table_name} AS
            SELECT * FROM read_csv_auto('{path}');
        """)
        print(f"Completed: {file}\n")

# 5. Verify tables
tables = con.execute("SHOW TABLES").fetchall()
print("Tables in DB:", tables)

# 6. Copy temporary DB to Google Drive
shutil.copy(temp_db_path, final_db_path)
print(f"✔ Database saved to Google Drive: {final_db_path}")


Mounted at /content/drive
Connected to temporary DuckDB: /content/data_temp.duckdb
Processing: purchases.csv -> Table: purchases


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Completed: purchases.csv

Processing: begin_inventory.csv -> Table: begin_inventory
Completed: begin_inventory.csv

Processing: end_inventory.csv -> Table: end_inventory
Completed: end_inventory.csv

Processing: sales.csv -> Table: sales


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Completed: sales.csv

Processing: vendor_invoice.csv -> Table: vendor_invoice
Completed: vendor_invoice.csv

Processing: purchase_prices.csv -> Table: purchase_prices
Completed: purchase_prices.csv

Tables in DB: [('begin_inventory',), ('end_inventory',), ('purchase_prices',), ('purchases',), ('sales',), ('vendor_invoice',)]
✔ Database saved to Google Drive: /content/drive/MyDrive/data/data.duckdb


In [2]:
# checking tables present in the database
tables = con.execute("SHOW TABLES").fetchall()
print("Tables in DB:", tables)


Tables in DB: [('begin_inventory',), ('end_inventory',), ('purchase_prices',), ('purchases',), ('sales',), ('vendor_invoice',)]


In [3]:
import pandas as pd

In [4]:
# Preview the first 100 rows from the 'sales' table to verify data ingestion
df = con.execute("SELECT * FROM sales LIMIT 100;").fetchdf()  # returns a pandas DataFrame


In [5]:
df

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1_HARDERSFIELD_1033,1,1033,Chivas Regal w/ 2 Glasses,750mL,1,29.99,29.99,2024-01-03,750.0,1,0.79,17035,PERNOD RICARD USA
96,1_HARDERSFIELD_1035,1,1035,Pinnacle Vodka Gift Pak,750mL + 3/,2,21.98,10.99,2024-01-07,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
97,1_HARDERSFIELD_1035,1,1035,Pinnacle Vodka Gift Pak,750mL + 3/,1,10.99,10.99,2024-01-09,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
98,1_HARDERSFIELD_1035,1,1035,Pinnacle Vodka Gift Pak,750mL + 3/,1,10.99,10.99,2024-01-10,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


In [6]:
# Retrieve the list of all tables in the DuckDB database (from the main schema)
tables_df = con.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema='main';
""").fetchdf()

tables = tables_df['table_name'].tolist()
print("Tables in DB:", tables)

Tables in DB: ['begin_inventory', 'end_inventory', 'purchases', 'purchase_prices', 'sales', 'vendor_invoice']


In [7]:
# Loop through all user-created tables in the database:
# - Print table name and record count
# - Display the first 5 rows of each table for a quick preview / sanity check

for table in tables:
  print('-'*50, f'{table}','-'*50)
  print('count of records:',con.execute(f"SELECT count(*) as cnt FROM {table}").fetchdf()['cnt'].values[0])
  display(con.execute(f"SELECT * FROM {table} limit 5").fetchdf())

-------------------------------------------------- begin_inventory --------------------------------------------------
count of records: 206529


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,startDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,8,12.99,2024-01-01
1,1_HARDERSFIELD_60,1,HARDERSFIELD,60,Canadian Club 1858 VAP,750mL,7,10.99,2024-01-01
2,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,6,36.99,2024-01-01
3,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,3,38.99,2024-01-01
4,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,6,34.99,2024-01-01


-------------------------------------------------- end_inventory --------------------------------------------------
count of records: 224489


,InventoryId,Store,City,Brand,Description,Size,onHand,Price,endDate
0,1_HARDERSFIELD_58,1,HARDERSFIELD,58,Gekkeikan Black & Gold Sake,750mL,11,12.99,2024-12-31
1,1_HARDERSFIELD_62,1,HARDERSFIELD,62,Herradura Silver Tequila,750mL,7,36.99,2024-12-31
2,1_HARDERSFIELD_63,1,HARDERSFIELD,63,Herradura Reposado Tequila,750mL,7,38.99,2024-12-31
3,1_HARDERSFIELD_72,1,HARDERSFIELD,72,No. 3 London Dry Gin,750mL,4,34.99,2024-12-31
4,1_HARDERSFIELD_75,1,HARDERSFIELD,75,Three Olives Tomato Vodka,750mL,7,14.99,2024-12-31


-------------------------------------------------- purchases --------------------------------------------------
count of records: 2372474


...

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.


-------------------------------------------------- sales --------------------------------------------------
count of records: 12825363


,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY


-------------------------------------------------- vendor_invoice --------------------------------------------------
count of records: 5543


,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,105,ALTAMAR BRANDS LLC,2024-01-04,8124,2023-12-21,2024-02-16,6,214.26,3.47,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
2,388,ATLANTIC IMPORTING COMPANY,2024-01-09,8169,2023-12-24,2024-02-16,5,106.60,4.61,None
3,480,BACARDI USA INC,2024-01-12,8106,2023-12-20,2024-02-05,10100,137483.78,2935.20,None
4,516,BANFI PRODUCTS CORP,2024-01-07,8170,2023-12-24,2024-02-12,1935,15527.25,429.20,None


In [8]:
# Display all records from the 'purchase_prices' table for VendorNumber 4466
display(con.execute(f"SELECT * FROM purchase_prices WHERE VendorNumber=4466 ").fetchdf())

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,5215,TGI Fridays Long Island Iced,12.99,1750mL,1750,1,9.41,4466,AMERICAN VINTAGE BEVERAGE
1,5255,TGI Fridays Ultimte Mudslide,12.99,1750mL,1750,1,9.35,4466,AMERICAN VINTAGE BEVERAGE
2,3140,TGI Fridays Orange Dream,14.99,1750mL,1750,1,11.19,4466,AMERICAN VINTAGE BEVERAGE


In [9]:
# Display all records from the 'purchases' table for VendorNumber 4466
display(con.execute(f"SELECT * FROM purchases WHERE VendorNumber=4466 ").fetchdf())

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
1,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
2,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
3,38_GOULCREST_5215,38,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-07,2024-01-19,2024-02-26,9.41,6,56.46,1
4,59_CLAETHORPES_5215,59,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8207,2023-12-27,2024-01-05,2024-01-19,2024-02-26,9.41,6,56.46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,81_PEMBROKE_5215,81,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-29,2025-01-04,2025-02-10,9.41,6,56.46,1
2188,62_KILMARNOCK_5255,62,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.35,5,46.75,1
2189,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-28,2025-01-04,2025-02-10,9.41,5,47.05,1
2190,6_GOULCREST_5215,6,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,13595,2024-12-20,2024-12-31,2025-01-04,2025-02-10,9.41,6,56.46,1


In [10]:
# Display all records from the 'vendor_invoice' table for VendorNumber 4466
display(con.execute(f"SELECT * FROM vendor_invoice WHERE VendorNumber=4466 ").fetchdf())

,VendorNumber,VendorName,InvoiceDate,PONumber,PODate,PayDate,Quantity,Dollars,Freight,Approval
0,4466,AMERICAN VINTAGE BEVERAGE,2024-01-07,8137,2023-12-22,2024-02-21,15,140.55,8.57,None
1,4466,AMERICAN VINTAGE BEVERAGE,2024-01-19,8207,2023-12-27,2024-02-26,335,3142.33,16.97,None
2,4466,AMERICAN VINTAGE BEVERAGE,2024-01-18,8307,2024-01-03,2024-02-18,41,383.35,1.99,None
3,4466,AMERICAN VINTAGE BEVERAGE,2024-01-27,8469,2024-01-14,2024-03-11,72,673.20,3.30,None
4,4466,AMERICAN VINTAGE BEVERAGE,2024-02-04,8532,2024-01-19,2024-03-15,79,740.21,3.48,None
5,4466,AMERICAN VINTAGE BEVERAGE,2024-02-09,8604,2024-01-24,2024-03-15,347,3261.37,17.61,None
6,4466,AMERICAN VINTAGE BEVERAGE,2024-02-17,8793,2024-02-05,2024-04-02,72,675.36,3.17,None
7,4466,AMERICAN VINTAGE BEVERAGE,2024-03-01,8892,2024-02-12,2024-03-28,117,1096.05,5.15,None
8,4466,AMERICAN VINTAGE BEVERAGE,2024-03-07,8995,2024-02-19,2024-04-02,129,1209.27,5.44,None
9,4466,AMERICAN VINTAGE BEVERAGE,2024-03-12,9033,2024-02-22,2024-04-16,147,1377.87,6.61,None


In [11]:
# Display all records from the 'sales' table for VendorNumber 4466
display(con.execute(f"SELECT * FROM sales WHERE VendorNo=4466 ").fetchdf())

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
1,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-12,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
2,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-15,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
3,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
4,1_HARDERSFIELD_5215,1,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-01-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9448,9_BLACKPOOL_5215,9,5215,TGI Fridays Long Island Iced,1.75L,1,12.99,12.99,2024-12-21,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9449,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-02,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9450,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-09,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE
9451,9_BLACKPOOL_5255,9,5255,TGI Fridays Ultimte Mudslide,1.75L,1,12.99,12.99,2024-12-23,1750.0,1,1.84,4466,AMERICAN VINTAGE BEVERAGE


In [12]:
# Load the entire 'purchases' table into a pandas DataFrame
# Display the raw data to inspect its structure
# Then, aggregate total Quantity and Dollars by Brand and PurchasePrice
df = con.execute("SELECT * FROM purchases").fetchdf()

display(df)

df.groupby(['Brand', 'PurchasePrice'])[['Quantity', 'Dollars']].sum()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,InventoryId,Store,Brand,Description,Size,VendorNumber,VendorName,PONumber,PODate,ReceivingDate,InvoiceDate,PayDate,PurchasePrice,Quantity,Dollars,Classification
0,69_MOUNTMEND_8412,69,8412,Tequila Ocho Plata Fresno,750mL,105,ALTAMAR BRANDS LLC,8124,2023-12-21,2024-01-02,2024-01-04,2024-02-16,35.71,6,214.26,1
1,30_CULCHETH_5255,30,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,4,37.40,1
2,34_PITMERDEN_5215,34,5215,TGI Fridays Long Island Iced,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-02,2024-01-07,2024-02-21,9.41,5,47.05,1
3,1_HARDERSFIELD_5255,1,5255,TGI Fridays Ultimte Mudslide,1.75L,4466,AMERICAN VINTAGE BEVERAGE,8137,2023-12-22,2024-01-01,2024-01-07,2024-02-21,9.35,6,56.10,1
4,76_DONCASTER_2034,76,2034,Glendalough Double Barrel,750mL,388,ATLANTIC IMPORTING COMPANY,8169,2023-12-24,2024-01-02,2024-01-09,2024-02-16,21.32,5,106.60,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372469,49_GARIGILL_22298,49,22298,Zorvino Vyds Sangiovese,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-28,2025-01-09,2025-02-06,8.06,12,96.72,2
2372470,1_HARDERSFIELD_19556,1,19556,Zorvino Bacca Z Blackberry,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-27,2025-01-09,2025-02-06,9.39,12,112.68,2
2372471,66_EANVERNESS_22297,66,22297,Zorvino Vyds Pearz,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-26,2025-01-09,2025-02-06,6.75,12,81.00,2
2372472,69_MOUNTMEND_19557,69,19557,Zorvino Fragole Z Strawberry,750mL,90058,ZORVINO VINEYARDS,13593,2024-12-19,2024-12-26,2025-01-09,2025-02-06,9.39,12,112.68,2


,,Quantity,Dollars
Brand,PurchasePrice,,
58,9.28,3550,32944.00
60,7.40,1633,12084.20
61,10.60,312,3307.20
62,28.67,3200,91744.00
63,30.46,2855,86963.30
...,...,...,...
90089,77.92,32,2493.44
90090,448.27,6,2689.62
90604,78.42,118,9253.56


In [13]:
# Load the entire 'purchase_prices' table into a pandas DataFrame
# Display the table to inspect vendor pricing data and its structure
df = con.execute("SELECT * FROM purchase_prices").fetchdf()

display(df)

,Brand,Description,Price,Size,Volume,Classification,PurchasePrice,VendorNumber,VendorName
0,58,Gekkeikan Black & Gold Sake,12.99,750mL,750,1,9.28,8320,SHAW ROSS INT L IMP LTD
1,62,Herradura Silver Tequila,36.99,750mL,750,1,28.67,1128,BROWN-FORMAN CORP
2,63,Herradura Reposado Tequila,38.99,750mL,750,1,30.46,1128,BROWN-FORMAN CORP
3,72,No. 3 London Dry Gin,34.99,750mL,750,1,26.11,9165,ULTRA BEVERAGE COMPANY LLP
4,75,Three Olives Tomato Vodka,14.99,750mL,750,1,10.94,7245,PROXIMO SPIRITS INC.
...,...,...,...,...,...,...,...,...,...
12256,44917,Ferreira 10-Yr Tawny Port,24.99,750mL,750,2,16.55,90024,VINILANDIA USA
12257,44944,Sanford Santa Rita Pnt Nr,22.99,750mL,750,2,14.93,4425,MARTIGNETTI COMPANIES
12258,45016,Neal One Lane Bridg Cab Svgn,93.99,750mL,750,2,61.43,10754,PERFECTA WINES
12259,46011,Folonari Pnt Nr Venezie,12.99,1500ml,1500,2,8.90,9744,FREDERICK WILDMAN & SONS


In [14]:
# Load the 'vendor_invoice' table into a pandas DataFrame
# Check the number of unique values in each column to understand data diversity and identify potential categorical variables
df = con.execute("SELECT * FROM vendor_invoice").fetchdf()

df.nunique()


,0
VendorNumber,126
VendorName,129
InvoiceDate,373
PONumber,5543
PODate,319
PayDate,382
Quantity,2895
Dollars,5226
Freight,4052
Approval,1


In [15]:
# Check the shape of the DataFrame (rows, columns) to understand the dataset size
df.shape


(5543, 10)

In [16]:
# Load the 'sales' table into a DataFrame for exploration.
# Display the raw data, then aggregate key sales metrics (dollars, price, quantity)
# grouped by Brand to analyze brand-level sales performance.
df = con.execute("SELECT * FROM sales").fetchdf()
display(df)
df.groupby(['Brand'])[['SalesDollars', 'SalesPrice', 'SalesQuantity']].sum()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,InventoryId,Store,Brand,Description,Size,SalesQuantity,SalesDollars,SalesPrice,SalesDate,Volume,Classification,ExciseTax,VendorNo,VendorName
0,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-01,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
1,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,2,32.98,16.49,2024-01-02,750.0,1,1.57,12546,JIM BEAM BRANDS COMPANY
2,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,16.49,16.49,2024-01-03,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
3,1_HARDERSFIELD_1004,1,1004,Jim Beam w/2 Rocks Glasses,750mL,1,14.49,14.49,2024-01-08,750.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
4,1_HARDERSFIELD_1005,1,1005,Maker's Mark Combo Pack,375mL 2 Pk,2,69.98,34.99,2024-01-09,375.0,1,0.79,12546,JIM BEAM BRANDS COMPANY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12825358,9_BLACKPOOL_984,9,984,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-17,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP
12825359,9_BLACKPOOL_984,9,984,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-21,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP
12825360,9_BLACKPOOL_984,9,984,Patron XO Cafe Liqueur,750mL,3,77.97,25.99,2024-12-23,750.0,1,2.36,9165,ULTRA BEVERAGE COMPANY LLP
12825361,9_BLACKPOOL_984,9,984,Patron XO Cafe Liqueur,750mL,1,25.99,25.99,2024-12-24,750.0,1,0.79,9165,ULTRA BEVERAGE COMPANY LLP


,SalesDollars,SalesPrice,SalesQuantity
Brand,,,
58,43341.54,28145.64,3446
60,18716.25,10720.79,1775
61,4364.88,363.74,312
62,119863.75,90154.51,3125
63,112249.22,88553.10,2778
...,...,...,...
90089,19078.41,5759.52,159
90090,9749.85,4549.93,15
90604,9119.24,2639.78,76


In [17]:
# Summarize total freight (shipping) cost charged by each vendor.
# This helps identify which suppliers incur higher transportation expenses.
freight_summary =  con.execute("SELECT VendorNumber, SUM(Freight) as FrieghtCost FROM vendor_invoice GROUP BY VendorNumber").fetchdf()
freight_summary

,VendorNumber,FrieghtCost
0,4466,793.91
1,388,211.74
2,480,89286.27
3,2396,30.98
4,1128,68601.68
...,...,...
121,90026,0.36
122,200,6.19
123,3551,26.63
124,201359,0.09


In [18]:
# Retrieve a single sample row from the 'purchases' table to inspect the column names and understand the table structure.
df = con.execute("SELECT * FROM purchases LIMIT 1").fetchdf()
df.columns



Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'VendorNumber',
       'VendorName', 'PONumber', 'PODate', 'ReceivingDate', 'InvoiceDate',
       'PayDate', 'PurchasePrice', 'Quantity', 'Dollars', 'Classification'],
      dtype='object')

In [19]:
# Retrieve a single sample row from the 'purchase_prices' table to inspect the column names and understand the table structure.
df = con.execute("SELECT * FROM purchase_prices LIMIT 1").fetchdf()
df.columns


Index(['Brand', 'Description', 'Price', 'Size', 'Volume', 'Classification',
       'PurchasePrice', 'VendorNumber', 'VendorName'],
      dtype='object')

In [20]:
# Join purchases transactions with purchase_price details to compute total quantity and spend per
# vendor-brand combination. This helps analyze vendor performance, pricing patterns,
# and high-value purchase segments.
con.execute("""
SELECT
  p.VendorNumber,
  p.VendorName,
  p.Brand,
  p.PurchasePrice,
  pp.Volume,
  pp.Price AS ActualPrice,
  SUM(p.Quantity) AS TotalPurchaseQuantity,
  SUM(p.Dollars) AS TotalPurchaseDollars
FROM purchases p
JOIN purchase_prices pp
  ON p.Brand = pp.Brand
WHERE p.PurchasePrice>0
GROUP BY
  p.VendorNumber,
  p.VendorName,
  p.Brand,
  p.PurchasePrice,
  pp.Volume,
  ActualPrice
ORDER BY TotalPurchaseDollars
""").fetchdf()


,VendorNumber,VendorName,Brand,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars
0,7245,PROXIMO SPIRITS INC.,3065,0.71,50,0.99,1.0,0.71
1,3960,DIAGEO NORTH AMERICA INC,6127,1.47,200,1.99,1.0,1.47
2,3924,HEAVEN HILL DISTILLERIES,9123,0.74,50,0.99,2.0,1.48
3,8004,SAZERAC CO INC,5683,0.39,50,0.49,6.0,2.34
4,9815,WINE GROUP INC,8527,1.32,750,4.99,2.0,2.64
...,...,...,...,...,...,...,...,...
10687,3960,DIAGEO NORTH AMERICA INC,3545,21.89,1750,29.99,138109.0,3023206.01
10688,3960,DIAGEO NORTH AMERICA INC,4261,16.17,1750,22.99,201682.0,3261197.94
10689,17035,PERNOD RICARD USA,8068,18.24,1750,24.99,187407.0,3418303.68
10690,4425,MARTIGNETTI COMPANIES,3405,23.19,1750,28.99,164038.0,3804041.22


In [21]:
# Retrieve a single sample row from the 'purchases' table to inspect the column names and understand the table structure.
df = con.execute("SELECT * FROM sales LIMIT 1").fetchdf()
df.columns

Index(['InventoryId', 'Store', 'Brand', 'Description', 'Size', 'SalesQuantity',
       'SalesDollars', 'SalesPrice', 'SalesDate', 'Volume', 'Classification',
       'ExciseTax', 'VendorNo', 'VendorName'],
      dtype='object')

In [23]:
# Aggregate sales data by Vendor and Brand to analyze revenue, sales volume,
# pricing totals, and excise tax contribution for each vendor–brand combination.
con.execute(""" SELECT
VendorNO,
Brand,
SUM(SalesDollars) AS TotalSalesDollars,
SUM(SalesQuantity) AS TotalSalesQuantity,
SUM(SalesPrice) AS TotalSalesPrice,
SUm(ExciseTax) AS TotalExciseTax
FROM sales
GROUP BY VendorNO, Brand
ORDER BY TotalSalesDollars
""").fetchdf()

,VendorNo,Brand,TotalSalesDollars,TotalSalesQuantity,TotalSalesPrice,TotalExciseTax
0,8004,5287,0.98,2.0,0.98,0.10
1,9206,2773,0.99,1.0,0.99,0.05
2,3924,9123,1.98,2.0,0.99,0.10
3,3252,3933,1.98,2.0,0.99,0.10
4,10050,3623,1.98,2.0,1.98,0.10
...,...,...,...,...,...,...
11267,3960,3545,4223107.62,135838.0,545778.28,249587.83
11268,3960,4261,4475972.88,200412.0,420050.01,368242.80
11269,17035,8068,4538120.60,187140.0,461140.15,343854.07
11270,4425,3405,4819073.49,160247.0,561512.37,294438.66


In [24]:
# Build the final vendor_sales_summary table using CTEs:
# 1. FreightSummary → Calculates total freight cost per vendor.
# 2. PurchaseSummary → Aggregates purchase quantities and spend per vendor–brand,
#    joining purchases data with purchase_price information.
# 3. SalesSummary → Aggregates sales metrics (quantity, revenue, price, excise tax) per vendor–brand.
# The final SELECT joins all summaries to create a unified vendor_sales_summary table
# combining purchase, sales, and freight cost metrics for analysis.
vendor_sales_summary = con.execute("""WITH FreightSummary AS(
  SELECT VendorNumber,
  SUM(Freight) AS TotalFreightCost
  FROM vendor_invoice
  GROUP BY VendorNumber
),
PurchaseSummary AS(
  SELECT
  p.VendorNumber,
  p.VendorName,
  p.Brand,
  p.Description,
  p.PurchasePrice,
  pp.Volume,
  pp.Price AS ActualPrice,
  SUM(Quantity) AS TotalPurchaseQuantity,
  SUM(Dollars) AS TotalPurchaseDollars
  FROM purchases p
  JOIN purchase_prices pp
  ON p.Brand = pp.Brand
  WHERE p.PurchasePrice > 0
  GROUP BY p.VendorNumber, p.VendorName, p.Brand, p.Description, p.PurchasePrice, pp.Volume, pp.Price
),
SalesSummary AS(
  SELECT
  VendorNo,
  Brand,
  SUM(SalesQuantity) AS TotalSalesQuantity,
  SUM(SalesDollars) AS TotalSalesDollars,
  SUM(SalesPrice) AS TotalSalesPrice,
  SUM(ExciseTax) AS TotalExciseTax
  FROM sales
  GROUP BY VendorNo, Brand
)
SELECT
ps.vendorNumber,
ps.VendorName,
ps.Brand,
ps.Description,
ps.purchasePrice,
ps.Volume,
ps.ActualPrice,
ps.TotalPurchaseQuantity,
ps.TotalPurchaseDollars,
fs.TotalFreightCost,
ss.TotalSalesQuantity,
ss.TotalSalesDollars,
ss.TotalSalesPrice,
ss.TotalExciseTax
FROM PurchaseSummary ps
LEFT JOIN SalesSummary ss
ON ps.VendorNumber = ss.VendorNo
AND ps.Brand = ss.Brand
LEFT JOIN FreightSummary fs
ON ps.VendorNumber = fs.VendorNumber
ORDER BY ps.TotalPurchaseDollars DESC
""").fetchdf()

display(vendor_sales_summary)

,VendorNumber,VendorName,Brand,Description,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars,TotalFreightCost,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,1750,36.99,145080.0,3811251.60,68601.68,142049.0,5101919.51,672819.31,260999.20
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,1750,28.99,164038.0,3804041.22,144929.24,160247.0,4819073.49,561512.37,294438.66
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,1750,24.99,187407.0,3418303.68,123780.22,187140.0,4538120.60,461140.15,343854.07
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,1750,22.99,201682.0,3261197.94,257032.07,200412.0,4475972.88,420050.01,368242.80
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,1750,29.99,138109.0,3023206.01,257032.07,135838.0,4223107.62,545778.28,249587.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,750,4.99,2.0,2.64,27100.41,5.0,15.95,10.96,0.55
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,50,0.49,6.0,2.34,50293.62,134.0,65.66,1.47,7.04
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,50,0.99,2.0,1.48,14069.87,2.0,1.98,0.99,0.10
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,200,1.99,1.0,1.47,257032.07,72.0,143.28,77.61,15.12


In [25]:
# Check the data types of all columns to understand variable types
# and ensure correct formatting before further analysis or modeling.
vendor_sales_summary.dtypes

,0
VendorNumber,int64
VendorName,object
Brand,int64
Description,object
PurchasePrice,float64
Volume,object
ActualPrice,float64
TotalPurchaseQuantity,float64
TotalPurchaseDollars,float64
TotalFreightCost,float64


In [26]:
# Check the number of missing values in each column to identify data quality issues
# and decide if cleaning or imputation is needed.
vendor_sales_summary.isnull().sum()

,0
VendorNumber,0
VendorName,0
Brand,0
Description,0
PurchasePrice,0
Volume,0
ActualPrice,0
TotalPurchaseQuantity,0
TotalPurchaseDollars,0
TotalFreightCost,0


In [27]:
# Check unique vendor names to detect issues like leading/trailing whitespaces,
# inconsistent casing, or duplicate vendor names that look similar.
vendor_sales_summary['VendorName'].unique()

array(['BROWN-FORMAN CORP          ', 'MARTIGNETTI COMPANIES',
       'PERNOD RICARD USA          ', 'DIAGEO NORTH AMERICA INC   ',
       'BACARDI USA INC            ', 'JIM BEAM BRANDS COMPANY    ',
       'MAJESTIC FINE WINES        ', 'ULTRA BEVERAGE COMPANY LLP ',
       'STOLI GROUP,(USA) LLC      ', 'PROXIMO SPIRITS INC.       ',
       'MOET HENNESSY USA INC      ', 'CAMPARI AMERICA            ',
       'SAZERAC CO INC             ', 'CONSTELLATION BRANDS INC   ',
       'M S WALKER INC             ', 'SAZERAC NORTH AMERICA INC. ',
       'PALM BAY INTERNATIONAL INC ', 'REMY COINTREAU USA INC     ',
       'SIDNEY FRANK IMPORTING CO  ', 'E & J GALLO WINERY         ',
       'WILLIAM GRANT & SONS INC   ', 'HEAVEN HILL DISTILLERIES   ',
       'DISARONNO INTERNATIONAL LLC', 'EDRINGTON AMERICAS         ',
       'CASTLE BRANDS CORP.        ', 'SOUTHERN WINE & SPIRITS NE ',
       'STE MICHELLE WINE ESTATES  ', 'TRINCHERO FAMILY ESTATES   ',
       'MHW LTD                    ', 'W

In [28]:
# Inspect unique product descriptions to identify formatting issues such as
# leading/trailing spaces, inconsistent naming, or duplicate descriptions.
vendor_sales_summary['Description'].unique()

array(['Jack Daniels No 7 Black', "Tito's Handmade Vodka",
       'Absolut 80 Proof', ..., 'Crown Royal Apple',
       'Concannon Glen Ellen Wh Zin', 'The Club Strawbry Margarita'],
      dtype=object)

In [29]:
# Convert 'Volume' column to float to ensure correct numeric type for calculations and analysis
vendor_sales_summary['Volume']=vendor_sales_summary['Volume'].astype('float64')

In [30]:
# Replace all missing values with 0 to avoid errors in calculations and ensure clean analysis
vendor_sales_summary.fillna(0,inplace=True)

In [31]:
# Remove leading and trailing whitespaces from 'VendorName' to ensure consistency.
vendor_sales_summary['VendorName'] = vendor_sales_summary['VendorName'].str.strip()

In [32]:
# Calculate Gross Profit for each vendor–brand by subtracting total purchase cost from total sales revenue
vendor_sales_summary['GrossProfit'] = vendor_sales_summary['TotalSalesDollars'] - vendor_sales_summary['TotalPurchaseDollars']

In [33]:
# Calculate Profit Margin (%) for each vendor–brand as (GrossProfit ÷ TotalSalesDollars) × 100
vendor_sales_summary['ProfitMargin'] = (vendor_sales_summary['GrossProfit']/vendor_sales_summary['TotalSalesDollars'])*100

In [34]:
# Calculate Stock Turnover ratio for each vendor–brand as Total Sales Quantity ÷ Total Purchase Quantity
# This indicates how efficiently inventory is being sold.
vendor_sales_summary['StockTurnOver'] = vendor_sales_summary['TotalSalesQuantity']/vendor_sales_summary['TotalPurchaseQuantity']

In [35]:
# Calculate Sales-to-Purchase Ratio for each vendor–brand as Total Sales Dollars ÷ Total Purchase Dollars
# This measures the revenue generated relative to purchase spend.
vendor_sales_summary['SaletoPurchaseRatio']=vendor_sales_summary['TotalSalesDollars']/vendor_sales_summary['TotalPurchaseDollars']

In [36]:
# Create a DuckDB cursor object to execute SQL queries and fetch results
cursor = con.cursor()

In [37]:
# Display all column names in the vendor_sales_summary DataFrame
# This helps verify the structure and ensure all expected metrics are present
vendor_sales_summary.columns

Index(['VendorNumber', 'VendorName', 'Brand', 'Description', 'PurchasePrice',
       'Volume', 'ActualPrice', 'TotalPurchaseQuantity',
       'TotalPurchaseDollars', 'TotalFreightCost', 'TotalSalesQuantity',
       'TotalSalesDollars', 'TotalSalesPrice', 'TotalExciseTax', 'GrossProfit',
       'ProfitMargin', 'StockTurnOver', 'SaletoPurchaseRatio'],
      dtype='object')

In [38]:
# List all tables currently available in the DuckDB database
# Useful for verifying which tables have been loaded or created
con.execute("SHOW TABLES").fetchdf()


,name
0,begin_inventory
1,end_inventory
2,purchase_prices
3,purchases
4,sales
5,vendor_invoice


In [39]:
# Display the full 'vendor_sales_summary' table to review the combined purchase, sales, and freight data
# This helps verify that all metrics have been calculated correctly
display(con.execute("SELECT * FROM vendor_sales_summary").fetchdf())

,VendorNumber,VendorName,Brand,Description,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars,TotalFreightCost,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,GrossProfit,ProfitMargin,StockTurnOver,SaletoPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,1750.0,36.99,145080.0,3811251.60,68601.68,142049.0,5101919.51,672819.31,260999.20,1290667.91,25.297693,0.979108,1.338647
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,1750.0,28.99,164038.0,3804041.22,144929.24,160247.0,4819073.49,561512.37,294438.66,1015032.27,21.062810,0.976890,1.266830
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,1750.0,24.99,187407.0,3418303.68,123780.22,187140.0,4538120.60,461140.15,343854.07,1119816.92,24.675786,0.998575,1.327594
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,1750.0,22.99,201682.0,3261197.94,257032.07,200412.0,4475972.88,420050.01,368242.80,1214774.94,27.139908,0.993703,1.372493
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,1750.0,29.99,138109.0,3023206.01,257032.07,135838.0,4223107.62,545778.28,249587.83,1199901.61,28.412764,0.983556,1.396897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,750.0,4.99,2.0,2.64,27100.41,5.0,15.95,10.96,0.55,13.31,83.448276,2.500000,6.041667
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,50.0,0.49,6.0,2.34,50293.62,134.0,65.66,1.47,7.04,63.32,96.436186,22.333333,28.059829
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,50.0,0.99,2.0,1.48,14069.87,2.0,1.98,0.99,0.10,0.50,25.252525,1.000000,1.337838
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,200.0,1.99,1.0,1.47,257032.07,72.0,143.28,77.61,15.12,141.81,98.974037,72.000000,97.469388


In [40]:
# Register the 'vendor_sales_summary' pandas DataFrame as a temporary DuckDB table named 'df_temp'
# This allows us to run SQL queries directly on the DataFrame within DuckDB
con.register("df_temp", vendor_sales_summary)


In [42]:
# Save the temporary DataFrame 'df_temp' as a permanent DuckDB table named 'vendor_sales_summary'
# This persists the cleaned and enriched vendor sales data for future queries and analysis
con.execute("""
CREATE TABLE vendor_sales_summary AS
SELECT * FROM df_temp
""")


In [43]:
# Display the persisted 'vendor_sales_summary' table from DuckDB
# This verifies that the DataFrame was successfully saved as a permanent table
display(con.execute("SELECT * FROM vendor_sales_summary").fetchdf())


,VendorNumber,VendorName,Brand,Description,PurchasePrice,Volume,ActualPrice,TotalPurchaseQuantity,TotalPurchaseDollars,TotalFreightCost,TotalSalesQuantity,TotalSalesDollars,TotalSalesPrice,TotalExciseTax,GrossProfit,ProfitMargin,StockTurnOver,SaletoPurchaseRatio
0,1128,BROWN-FORMAN CORP,1233,Jack Daniels No 7 Black,26.27,1750.0,36.99,145080.0,3811251.60,68601.68,142049.0,5101919.51,672819.31,260999.20,1290667.91,25.297693,0.979108,1.338647
1,4425,MARTIGNETTI COMPANIES,3405,Tito's Handmade Vodka,23.19,1750.0,28.99,164038.0,3804041.22,144929.24,160247.0,4819073.49,561512.37,294438.66,1015032.27,21.062810,0.976890,1.266830
2,17035,PERNOD RICARD USA,8068,Absolut 80 Proof,18.24,1750.0,24.99,187407.0,3418303.68,123780.22,187140.0,4538120.60,461140.15,343854.07,1119816.92,24.675786,0.998575,1.327594
3,3960,DIAGEO NORTH AMERICA INC,4261,Capt Morgan Spiced Rum,16.17,1750.0,22.99,201682.0,3261197.94,257032.07,200412.0,4475972.88,420050.01,368242.80,1214774.94,27.139908,0.993703,1.372493
4,3960,DIAGEO NORTH AMERICA INC,3545,Ketel One Vodka,21.89,1750.0,29.99,138109.0,3023206.01,257032.07,135838.0,4223107.62,545778.28,249587.83,1199901.61,28.412764,0.983556,1.396897
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,9815,WINE GROUP INC,8527,Concannon Glen Ellen Wh Zin,1.32,750.0,4.99,2.0,2.64,27100.41,5.0,15.95,10.96,0.55,13.31,83.448276,2.500000,6.041667
10688,8004,SAZERAC CO INC,5683,Dr McGillicuddy's Apple Pie,0.39,50.0,0.49,6.0,2.34,50293.62,134.0,65.66,1.47,7.04,63.32,96.436186,22.333333,28.059829
10689,3924,HEAVEN HILL DISTILLERIES,9123,Deep Eddy Vodka,0.74,50.0,0.99,2.0,1.48,14069.87,2.0,1.98,0.99,0.10,0.50,25.252525,1.000000,1.337838
10690,3960,DIAGEO NORTH AMERICA INC,6127,The Club Strawbry Margarita,1.47,200.0,1.99,1.0,1.47,257032.07,72.0,143.28,77.61,15.12,141.81,98.974037,72.000000,97.469388


In [44]:
# Export the 'vendor_sales_summary' DuckDB table to a CSV file named 'vendor_sales_summary.csv'
# The CSV will include column headers and use a comma as the delimiter,

con.execute("""
    COPY vendor_sales_summary
    TO 'vendor_sales_summary.csv'
    (HEADER, DELIMITER ',');
""")
